In [1]:
# imports


import os
import requests
import json

In [55]:
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from openai import OpenAI
from IPython.display import Markdown, display, update_display

In [3]:
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

In [4]:
# API 키 확인
if api_key and api_key.startswith("sk-proj") and len(api_key) >10:
    print('API key looks good so far')
else:
    print("API키에 문제가 있는 것 같습니다.")



API key looks good so far


In [5]:
MODEL = 'gpt-4o-mini'
openai = OpenAI()
# openai API model 설정

In [6]:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}


In [31]:
class Website:

    def __init__(self, url):
        self.url = url

        response = requests.get(url, headers=headers)

        self.body = response.content #response의 내용을 할당

        soup = BeautifulSoup(self.body, 'html.parser')
        
        self.title = soup.title.string if soup.title else "No title found"

        if soup.body:
            for irrelevant in soup.body(["script", 'style', 'img', 'input']):
                irrelevant.decompose()
                # .decompose(): Beautiful Soup의 Tag 객체가 제공하는 메서드로, 
                # 해당 태그를 문서 트리에서 완전히 제거합니다. 
                # 제거된 태그와 그 하위 요소들은 더 이상 soup 객체에서 접근할 수 없게 됩니다.

        # soup : Beautiful Soup으로 파싱된 HTML 또는 XML 문서 전체를 나타내는 객체입니다.

        # .body: soup 객체의 속성 중 하나로, HTML 문서의 <body>태그와 그 내용을 담고 있는Tag객체를 반환합니다. 
        #         만약 HTML 문서에&lt;body> 태그가 없다면 None을 반환할 수 있습니다.

            self.text = soup.body.get_text(separator="\n", strip=True)

        else:
            self.text = "No body found"
        
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [32]:
lge = Website("https://www.lge.co.kr/")
lge.links

['/',
 '#none',
 '/my-page',
 '/company/main',
 '/business',
 '#none',
 '#',
 '#',
 '#',
 '#',
 '#',
 '#',
 '#',
 '#',
 '#',
 '#',
 'https://www.lge.co.kr/tvs',
 'https://www.lge.co.kr/tvs?subCateId=CT50000028',
 'https://www.lge.co.kr/tvs?subCateId=CT50041000',
 'https://www.lge.co.kr/tvs?subCateId=CT50000029',
 'https://www.lge.co.kr/tvs?subCateId=CT50000030',
 'https://www.lge.co.kr/tvs?subCateId=CT50000026',
 'https://www.lge.co.kr/tvs?subCateId=CT50029007',
 'https://www.lge.co.kr/tvs?subCateId=CT50220000',
 'https://www.lge.co.kr/stan-by-me',
 'https://www.lge.co.kr/projectors',
 'https://www.lge.co.kr/projectors?subCateId=CT50000033',
 'https://www.lge.co.kr/projectors?subCateId=CT50256007',
 'https://www.lge.co.kr/projectors?subCateId=CT50308000',
 'https://www.lge.co.kr/projectors?subCateId=CT50077001',
 'https://www.lge.co.kr/home-audio',
 'https://www.lge.co.kr/home-audio?subCateId=CT50000044',
 'https://www.lge.co.kr/home-audio?subCateId=CT50000040',
 'https://www.lge.co.kr

In [33]:
link_system_prompt = "당신에게 웹페이지에서 발견된 링크들의 목록을 전달할 것입니다.\
    당신은 어떤 링크를 회사에 관한 브로슈어에 포함하는 것이 가장 관련성이 높은지 결정할 수 있습니다.\
    링크들은 about 페이지 또는 회사 페이지 또는 경력/ 채용정보 등의 링크 입니다"

link_system_prompt += "당신은 아래의 예시와 같이 JSON 형식으로 답변해야 합니다"
link_system_prompt += """
    {
        "links": [
            {"type": "about page", "url":"https://full.url/about"}
            {"type": "company page", "url":"'https://another.full.url/company"}
            ]
    }
    """

In [34]:
print(link_system_prompt)

당신에게 웹페이지에서 발견된 링크들의 목록을 전달할 것입니다.    당신은 어떤 링크를 회사에 관한 브로슈어에 포함하는 것이 가장 관련성이 높은지 결정할 수 있습니다.    링크들은 about 페이지 또는 회사 페이지 또는 경력/ 채용정보 등의 링크 입니다당신은 아래의 예시와 같이 JSON 형식으로 답변해야 합니다
    {
        "links": [
            {"type": "about page", "url":"https://full.url/about"}
            {"type": "company page", "url":"'https://another.full.url/company"}
            ]
    }
    


In [35]:
def get_links_user_prompt(website):
    user_prompt = f"여기에 웹사이트({website.url})의 링크 목록이 있습니다."
    user_prompt += f"링크들은 다음과 같습니다: {website.links}, JSON 형식의 전체 https URL로 응답하세요.\서비스 약관, 개인정보 보호정책, 이메일 링크는 포함하지 마세요.\n"
    user_prompt += "링크(일부는 상대적 링크일 수 있음):\n"
    user_prompt += "\n".join(website.links)
    
    return user_prompt

In [45]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = MODEL,
        messages = [
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
        ],
        response_format= {"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [46]:
lge = Website("https://www.lge.co.kr/")
lge.links

['/',
 '#none',
 '/my-page',
 '/company/main',
 '/business',
 '#none',
 '#',
 '#',
 '#',
 '#',
 '#',
 '#',
 '#',
 '#',
 '#',
 '#',
 'https://www.lge.co.kr/tvs',
 'https://www.lge.co.kr/tvs?subCateId=CT50000028',
 'https://www.lge.co.kr/tvs?subCateId=CT50041000',
 'https://www.lge.co.kr/tvs?subCateId=CT50000029',
 'https://www.lge.co.kr/tvs?subCateId=CT50000030',
 'https://www.lge.co.kr/tvs?subCateId=CT50000026',
 'https://www.lge.co.kr/tvs?subCateId=CT50029007',
 'https://www.lge.co.kr/tvs?subCateId=CT50220000',
 'https://www.lge.co.kr/stan-by-me',
 'https://www.lge.co.kr/projectors',
 'https://www.lge.co.kr/projectors?subCateId=CT50000033',
 'https://www.lge.co.kr/projectors?subCateId=CT50256007',
 'https://www.lge.co.kr/projectors?subCateId=CT50308000',
 'https://www.lge.co.kr/projectors?subCateId=CT50077001',
 'https://www.lge.co.kr/home-audio',
 'https://www.lge.co.kr/home-audio?subCateId=CT50000044',
 'https://www.lge.co.kr/home-audio?subCateId=CT50000040',
 'https://www.lge.co.kr

In [47]:
print(get_links("https://www.lge.co.kr/"))

{'links': [{'type': 'company page', 'url': 'https://www.lge.co.kr/company/main'}, {'type': 'business page', 'url': 'https://www.lge.co.kr/business'}, {'type': 'about page', 'url': 'https://www.lge.co.kr/story/history'}]}


### make brochure

In [48]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links", links)
    
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()

    return result


In [49]:
print(get_all_details("https://www.lge.co.kr/"))

Found links {'links': [{'type': 'company page', 'url': 'https://www.lge.co.kr/company/main'}, {'type': 'business page', 'url': 'https://www.lge.co.kr/business'}]}
Landing page:
Webpage Title:
LGE.COM | LG전자
Webpage Contents:
회사소개
사업자몰
제품/소모품
TV/오디오
PC/모니터
주방가전
생활가전
에어컨/에어케어
뷰티/의료기기
케어용품/소모품
LG SIGNATURE
LG Objet Collection
LG UP 가전
TV
올레드
QNED
나노셀
울트라 HD
일반 LED
라이프스타일 스크린
TV+사운드바
스탠바이미
프로젝터
시네빔
시네빔 세트
시네빔 액세서리
프로빔(상업용)
오디오
블루투스 이어폰
무선 스피커
사운드바
오디오/플레이어
상업용 디스플레이
올레드 사이니지
전자칠판
단독형사이니지
비디오월
LED 사이니지
원퀵
키오스크
스트레치
마인드웰니스
brid.zzz
#
갤러리디자인
#
홈 엔터테인먼트
#
캠핑영화관
스탠바이미2 런칭 혜택
2025 LG TV 기획전
스탠바이미 사용가이드
TV  구매가이드
올레드 TV
제품 라인업
TV 스펙 가전위키
올레드 TV FAQ
시네빔 큐브  FAQ
노트북
그램 Pro
그램 Pro 360
그램
울트라 PC
노트북 세트
Intel Core Ultra
Copilot+ PC
일체형/데스크톱/태블릿
일체형 PC
데스크톱
태블릿 기타
모니터
게이밍모니터
스마트모니터
스마트모니터 스윙
UHD모니터
와이드모니터
PC모니터
렛츠 그램
#
게임
#
재택
LG gram Pro 기획전
스마트모니터 스윙 런칭
노트북  구매가이드
애로우레이크
장점 알아보기
노트북 스펙 가전위키
그램 고객 리뷰
그램 Style 고객 리뷰
울트라기어 게이밍모니터
고객 리뷰
냉장고
STEM
상냉장/하냉동
양문형
일반형
페어설치키트
냉장고 세트
컨버터블 패키지
냉장전용고
냉동전용고
김치냉장고
페어

In [50]:
system_prompt = "회사 웹사이트의 여러 관련 페이지 내용을 분석하고 잠재 고객, 투자자, 신입 사원을 위한 회사 소개 간략 브로셔를 제작하는 어시스턴트입니다. 마크다운 형식으로 작성해 주세요. 회사 문화, 고객, 경력에 대한 세부 정보를 포함해야 합니다."

In [53]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"당신이 보고 있는 회사의 이름은{company_name}입니다\n"
    user_prompt += f"다음은 랜딩 페이지와 기타 관련 페이지의 내용입니다. 이 정보를 사용하여 마크다운 형식으로 회사의 짧은 브로셔를 작성하세요.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [54]:
get_brochure_user_prompt("LG Electronics", "https://www.lge.co.kr/")

Found links {'links': [{'type': 'company page', 'url': 'https://www.lge.co.kr/company/main'}, {'type': 'about page', 'url': 'https://www.lge.co.kr/business'}, {'type': 'career page', 'url': 'https://www.lge.co.kr/business/registration'}]}


'당신이 보고 있는 회사의 이름은LG Electronics입니다\n다음은 랜딩 페이지와 기타 관련 페이지의 내용입니다. 이 정보를 사용하여 마크다운 형식으로 회사의 짧은 브로셔를 작성하세요.\nLanding page:\nWebpage Title:\nLGE.COM | LG전자\nWebpage Contents:\n회사소개\n사업자몰\n제품/소모품\nTV/오디오\nPC/모니터\n주방가전\n생활가전\n에어컨/에어케어\n뷰티/의료기기\n케어용품/소모품\nLG SIGNATURE\nLG Objet Collection\nLG UP 가전\nTV\n올레드\nQNED\n나노셀\n울트라 HD\n일반 LED\n라이프스타일 스크린\nTV+사운드바\n스탠바이미\n프로젝터\n시네빔\n시네빔 세트\n시네빔 액세서리\n프로빔(상업용)\n오디오\n블루투스 이어폰\n무선 스피커\n사운드바\n오디오/플레이어\n상업용 디스플레이\n올레드 사이니지\n전자칠판\n단독형사이니지\n비디오월\nLED 사이니지\n원퀵\n키오스크\n스트레치\n마인드웰니스\nbrid.zzz\n#\n갤러리디자인\n#\n홈 엔터테인먼트\n#\n캠핑영화관\n스탠바이미2 런칭 혜택\n2025 LG TV 기획전\n스탠바이미 사용가이드\nTV  구매가이드\n올레드 TV\n제품 라인업\nTV 스펙 가전위키\n올레드 TV FAQ\n시네빔 큐브  FAQ\n노트북\n그램 Pro\n그램 Pro 360\n그램\n울트라 PC\n노트북 세트\nIntel Core Ultra\nCopilot+ PC\n일체형/데스크톱/태블릿\n일체형 PC\n데스크톱\n태블릿 기타\n모니터\n게이밍모니터\n스마트모니터\n스마트모니터 스윙\nUHD모니터\n와이드모니터\nPC모니터\n렛츠 그램\n#\n게임\n#\n재택\nLG gram Pro 기획전\n스마트모니터 스윙 런칭\n노트북  구매가이드\n애로우레이크\n장점 알아보기\n노트북 스펙 가전위키\n그램 고객 리뷰\n그램 Style 고객 리뷰\n울트라기어 게이밍모니터\n고객 리뷰\n냉장고\nSTEM\n상냉장/하냉동\n양문형\n일반

In [56]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [ ]:
create_brochure("LG Electronics", "https://www.lge.co.kr/")

In [ ]:
def create_stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model= MODEL,
        messages=[
            {"role" : "system", "content" : system_prompt},
            {"role" : "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
        stream = True
    )

    response = ""
    display_handle = display(Markdown(""), display_id = True)
    #display_handle: IPython.display 모듈의 display() 함수를 사용하여 Markdown 형식의 내용을 화면에 표시합니다.
    # 이것은 Markdown이라는 클래스의 객체를 생성하는 부분입니다. Markdown 클래스는 인자로 받은 문자열을 마크다운 형식으로 해석하여 표시할 수 있는 객체를 만듭니다.
    # 여기서는 인자로 빈 문자열인 ""를 주고 있으므로, 처음에는 아무 내용도 없는 빈 마크다운 영역을 만듭니다.
    # display_id를 True로 설정하면, 이 display 호출로 생성된 출력 영역에 **고유한 식별자(ID)**가 부여됩니다.
    #이 식별자를 통해 나중에 다른 코드를 실행할 때 새로운 출력 영역을 만드는 대신, 이전에 생성된 이 특정 출력 영역의 내용을 업데이트할 수 있게 됩니다. 
    # 이는 스트리밍이나 동적으로 변하는 내용을 표시할 때 매우 유용합니다.
    for chunk in stream:
    # steam은 openai API의 응답
        response += chunk.choices[0].delta.content or ''
        # 객체 안에 있는 경로를 따라가서 응답의 실제 텍스트 내용을 가져옵니다. (이런 구조는 보통 OpenAI API의 스트리밍 응답에서 볼 수 있습니다.)
        # or ''은 추출한 내용이 None인 경우 오류가 나지 않도록 빈 문자열을 대신사용하겠다는 내용
        response = response.replace("```","").replace("markdown", "")
        # 이 작업들은 아마도 API 응답에 포함될 수 있는 마크다운 관련 기호들을 화면에 표시하기 전에 정리하기 위해 수행되는 것으로 보입니다.
        # .replace("```",""): 코드 블록을 나타내는 트리플 백틱(```)을 모두 찾아서 제거합니다.
        # .replace("markdown", ""): "markdown"이라는 단어를 모두 찾아서 제거합니다.
        update_display(Markdown(response), display_id=display_handle.display_id)

In [59]:
create_stream_brochure("LG Electronics", "https://www.lge.co.kr/")

Found links {'links': [{'type': 'company page', 'url': 'https://www.lge.co.kr/company/main'}, {'type': 'careers page', 'url': 'https://www.lge.co.kr/business'}, {'type': 'about page', 'url': 'https://www.lge.co.kr/story/history'}, {'type': 'about page', 'url': 'https://www.lge.co.kr/story/lge-news'}]}



# LG Electronics 소개 브로셔

## 회사 개요
LG Electronics는 혁신적인 기술과 디자인으로 세계적인 소비자 가전 및 전자제품을 제조하는 글로벌 리더입니다. 우리의 제품군은 TV, 노트북, 가전제품, 공기청정기 등 다양한 분야를 아우릅니다.

## 회사 문화
LG Electronics는 직원들이 창의성과 혁신을 발휘할 수 있는 환경을 제공합니다. 우리 팀은 서로 존중하고 협력하며, 지속적인 학습과 성장을 추구합니다. 고객의 만족을 최우선으로 여기며, 사회적 책임을 다하고자 노력합니다.

## 고객 중심의 접근
LG Electronics는 고객의 니즈에 귀 기울이며, 제품 개발 및 서비스에서 고객의 피드백을 적극 반영합니다. 다양한 제품 라인업과 맞춤형 서비스를 통해, 고객의 일상 생활을 보다 편리하고 쾌적하게 만들어 나가고 있습니다. 

## 경력 기회
LG Electronics는 창의적이고 열정적인 인재를 찾습니다. 다양한 분야의 전문가들과 함께 성장하며, 국내외에서 활발한 경력 개발 기회를 제공합니다. 신입사원에게는 체계적인 교육과 멘토링을 통해 잠재력을 최대한 발휘할 수 있도록 지원합니다.

## 결론
LG Electronics와 함께 더 나은 미래를 설계해 보세요. 혁신, 고객 중심, 그리고 지속 가능한 사회에 기여하는 기업의 일원이 되실 수 있는 기회를 제공합니다.

